In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
    
import seaborn as sns
from scipy import stats


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/can-i-be-an-influencer/sample_submission.csv
/kaggle/input/can-i-be-an-influencer/test_colors.csv
/kaggle/input/can-i-be-an-influencer/test_keywords.csv
/kaggle/input/can-i-be-an-influencer/train.csv
/kaggle/input/can-i-be-an-influencer/train_colors.csv
/kaggle/input/can-i-be-an-influencer/test.csv
/kaggle/input/can-i-be-an-influencer/train_keywords.csv


In [2]:
df = pd.read_csv("/kaggle/input/can-i-be-an-influencer/train.csv")
test_df = pd.read_csv("/kaggle/input/can-i-be-an-influencer/test.csv")
sub = pd.read_csv("/kaggle/input/can-i-be-an-influencer/sample_submission.csv")

df_colors = pd.read_csv("/kaggle/input/can-i-be-an-influencer/train_colors.csv")
df_keywords = pd.read_csv("/kaggle/input/can-i-be-an-influencer/train_keywords.csv")

dft_colors = pd.read_csv("/kaggle/input/can-i-be-an-influencer/test_colors.csv")
dft_keywords = pd.read_csv("/kaggle/input/can-i-be-an-influencer/test_keywords.csv")

In [3]:
obs_day_mapper = {
'Friday':5, 'Tuesday':2, 'Sunday':0, 'Monday':1, 'Wednesday':3, 'Thursday':4,
       'Saturday':6
}

df['obs_day'] = df['obs_day'].map(obs_day_mapper)
test_df['obs_day'] = test_df['obs_day'].map(obs_day_mapper)

In [4]:
df['colors_mean_sore'] = df['image_id'].map(df_colors.groupby('image_id')['color_score'].mean())
df['colors_freq'] = df['image_id'].map(df_colors['image_id'].value_counts())
df['keywords_freq'] = df['image_id'].map(df_keywords['image_id'].value_counts())

cat_fts = [c for c in df.select_dtypes('object').columns.tolist() if c not in ["image_id", "stats_downloads", 'is_train', 'description1', 'description2']]
df[cat_fts] = df[cat_fts].apply(lambda x: pd.factorize(x)[0])

# ------------

test_df['colors_mean_sore'] = test_df['image_id'].map(dft_colors.groupby('image_id')['color_score'].mean())
test_df['colors_freq'] = test_df['image_id'].map(dft_colors['image_id'].value_counts())
test_df['keywords_freq'] = test_df['image_id'].map(dft_keywords['image_id'].value_counts())

cat_fts = [c for c in test_df.select_dtypes('object').columns.tolist() if c not in ["image_id", "stats_downloads", 'is_train', 'description1', 'description2']]
test_df[cat_fts] = test_df[cat_fts].apply(lambda x: pd.factorize(x)[0])

In [5]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
df = df.select_dtypes(include=numerics)

# remove useless cols
df.drop(["latitude", "longitude"], axis=1, inplace=True)
test_df.drop(["latitude", "longitude"], axis=1, inplace=True)

In [6]:
cols = df.loc[:, ~df.columns.isin(["image_category"])].columns

In [7]:
cols = df.loc[:, df.columns != 'stats_downloads'].columns
test_df = test_df[cols]

In [8]:
df = df.fillna(df.mean())
test_df = test_df.fillna(test_df.mean())

In [9]:
X_train = df.loc[:, df.columns != 'stats_downloads']
y_train = df["stats_downloads"]

In [10]:
from sklearn.preprocessing import StandardScaler

transformer = StandardScaler()
X_train = transformer.fit_transform(X_train)
X_test = transformer.transform(test_df)

In [11]:
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(max_depth = 100, oob_score = True, n_jobs = -1,random_state =50, max_features = "auto", min_samples_leaf = 30)

# from sklearn.linear_model import Ridge
# regr = Ridge()

regr.fit(X_train, y_train)
y_pred = regr.predict(X_train)

In [12]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_train, y_pred, squared=False)

33091.31929510119

In [13]:
res = regr.predict(X_test)
sub["stats_downloads"] = res
sub.to_csv("submission.csv", index=False)

In [14]:
print(min(res), max(res))

1194.2077291716885 60851.284221167836
